# **Analisis Spasial Deteksi Banjir Menggunakan Sentinel-1 SAR Berbasis Perubahan Backscatter di Kabupaten Karawang**

Deteksi banjir dilakukan menggunakan Sentinel-1 SAR melalui analisis perubahan backscatter sebelum dan sesudah kejadian. Hasil menunjukkan penurunan backscatter yang signifikan pada wilayah dataran rendah, yang diinterpretasikan sebagai zona genangan dengan tingkat keparahan bervariasi. Pola terfragmentasi pada hasil deteksi merupakan karakteristik alami data SAR akibat speckle noise dan heterogenitas permukaan

In [1]:
# 1. Install library yang dibutuhkan
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00


In [2]:
import ee
import geemap

# Gunakan Project ID kamu
project_id = 'project-eksplorasi'

try:
    ee.Initialize(project=project_id)
    print("Inisialisasi Berhasil!")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=project_id)

In [3]:
roi = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM1_NAME', 'Jawa Barat')) \
    .filter(ee.Filter.eq('ADM2_NAME', 'Karawang'))

In [4]:
# =============================
# 3️⃣ PREPROCESS SENTINEL-1
# =============================
def preprocess_s1(img):
    vv = img.select('VV')
    vv_db = ee.Image(10).multiply(vv.log10()).rename('VV_db')
    return vv_db.focal_median(3).copyProperties(img, img.propertyNames())


In [5]:
# =============================
# 4️⃣ SENTINEL-1 COLLECTION
# =============================
s1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi.geometry())
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .map(preprocess_s1)
)

In [6]:
# =============================
# 5️⃣ BEFORE & AFTER IMAGE
# =============================
before = (
    s1.filterDate('2023-01-01', '2023-02-15')
    .median()
    .select('VV_db')
    .clip(roi.geometry())
)

after = (
    s1.filterDate('2023-03-01', '2023-04-15')
    .median()
    .select('VV_db')
    .clip(roi.geometry())
)

In [7]:
# =============================
# 6️⃣ DATA PENDUKUNG
# =============================
slope = ee.Terrain.slope(
    ee.Image("USGS/SRTMGL1_003")
).clip(roi.geometry())

In [8]:
# =============================
# 7️⃣ DETEKSI BANJIR (FINAL)
# =============================
diff = after.subtract(before)

flood = (
    diff.lt(-1.2)        # penurunan backscatter
    .And(after.lt(-14))  # air dangkal / genangan
    .And(slope.lt(7))    # dataran relatif landai
).selfMask().rename('flood')

In [9]:
# =============================
# 8️⃣ HITUNG LUAS BANJIR (HEKTAR)
# =============================
area = flood.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi.geometry(),
    scale=10,
    maxPixels=1e13
)

area_ha = ee.Number(area.get('flood')).divide(10000)
print("Estimasi luas banjir (ha):", area_ha.getInfo())

Estimasi luas banjir (ha): 157.77432830024867


In [10]:
# =============================
# 🔟 FINAL VISUALISASI
# =============================
Map = geemap.Map(center=[-6.32, 107.30], zoom=10)
Map.add_basemap("HYBRID")

# 1️⃣ SAR AFTER (konteks observasi)
Map.addLayer(
    after,
    {'min': -25, 'max': 0},
    'Sentinel-1 SAR (After Flood)',
    False
)

# 2️⃣ Perubahan Backscatter (makna ilmiah)
Map.addLayer(
    diff,
    {
        'min': -4,
        'max': 2,
        'palette': ['darkred', 'red', 'orange', 'white']
    },
    'Backscatter Change (After - Before)',
    False
)

# 3️⃣ Flood Severity (zonasi genangan)
severity = (
    diff.where(diff.lt(-3), 3)
        .where(diff.gte(-3).And(diff.lt(-2)), 2)
        .where(diff.gte(-2).And(diff.lt(-1)), 1)
        .rename('severity')
)

Map.addLayer(
    severity.selfMask(),
    {
        'min': 1,
        'max': 3,
        'palette': ['#ffd166', '#f8961e', '#c1121f']
    },
    'Flood Severity'
)

# 4️⃣ Flooded Area Final (bersih & tegas)
Map.addLayer(
    flood,
    {
        'palette': ['#c1121f'],
        'opacity': 0.8
    },
    'Detected Flooded Area'
)

Map.addLayerControl()
Map


Map(center=[-6.32, 107.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…